# NoSQL - Not only SQL

Snowflake supports the following kinds of data:

- Structured data (AKA "SQL data") — such as rows and columns in a table — follows a strict tabular schema.

- Semi-structured data — such as a JSON file or an XML file — has a flexible schema.

- Unstructured data — such as a document, image, or audio file — has no inherent schema.


## Unstructured data

Supports the following actions:

- Securely access data files located in cloud storage.
- Share file access URLs with collaborators and partners.
- Load file access URLs and other file metadata into Snowflake tables.
- Process unstructured data.
- Load unstructured data with Document AI


### URL Types

#### Scoped URL

- Permits temporary access to a staged file without granting privileges to the stage.
- The URL expires when the persisted query result period ends.

#### File URL - Stage File URL 

- URL that identifies the database, schema, stage, and file path to a set of files. 
- Does not expire.
- A role that has sufficient privileges on the stage can access the files.

Use case: You can pass this URL to Snowflake’s files REST API to download the object.

#### Pre-signed URL

- Simple HTTPS URL used to access a file via a web browser.
- A file is temporarily accessible to users via this URL using a pre-signed access token.
- The expiration time for the access token is configurable.


In [ ]:
%%sql -r dataframe_7
use sf_cert_prep.public;

In [ ]:

CREATE OR REPLACE STAGE sf_cert_prep.public.my_images_stage
DIRECTORY = (ENABLE = TRUE)
ENCRYPTION  = (TYPE = 'SNOWFLAKE_SSE')
  ;

-- please upload some images in your stage, I tried to automate, but trial accounts dont allow external integration.


In [ ]:
%%sql -r dataframe_13
alter stage sf_cert_prep.public.my_images_stage refresh;

In [ ]:
%%sql -r dataframe_5
list @sf_cert_prep.public.my_images_stage;

In [ ]:
%%sql -r dataframe_6

SELECT *
FROM DIRECTORY(@sf_cert_prep.public.my_images_stage)  -- exposes RELATIVE_PATH, SIZE, LAST_MODIFIED, FILE_URL, etc.
ORDER BY RELATIVE_PATH;


In [ ]:
%%sql -r dataframe_9
-- Build a stable file URL (must know relative path)
SELECT BUILD_STAGE_FILE_URL('@sf_cert_prep.public.my_images_stage', 'my_images_stage/1.jpg') AS file_url;

In [ ]:
SELECT BUILD_SCOPED_FILE_URL('@sf_cert_prep.public.my_images_stage', '1.jpg') AS scoped_url;

In [ ]:
%%sql -r dataframe_11
-- 10-minute presigned link
SELECT GET_PRESIGNED_URL('@sf_cert_prep.public.my_images_stage', '1.jpg', 600) AS presigned_url;

## Directory tables

is an implicit object layered on a stage (not a separate database object) 

- Both external (external cloud storage) and internal (Snowflake) stages support directory tables
- Query a list of all the unstructured files on a stage.including the size, a timestamp of when it was last modified, and its Snowflake file URL.
- Create views of unstructured data. You can join a directory table with a Snowflake table that contains additional data and metadata about unstructured files to see unstructured files and their related data in a single view.
- Construct a file processing pipeline. You can use a directory table with the Snowpark API or external functions to create a file processing pipeline.